# <p style="text-align:center;"> Building a Network of Landownership in Kenya </p>

### <p style="text-align:center;"> A Prototype for Extracting Structured Text Data from the Kenya Gazettes </p>

**Code for Africa** (CfA): Allan Cheboi, Clemence Kyara, Jacopo Ottaviani, Justin Arenstein, Robin Kiplangat

**The World Resources Institute** (WRI): Kathleen Buckingham

**Data Science + Social Good (DSSG) @ Stanford Fellows**: Thea Rossman, Tsion Tesfaye, and Robbie Thompson

**DSSG faculty and technical mentors**: Mahnaz Roshanaei, Mike Sklar, Ben Stenhaug, Qijia Jiang, Michael Baiocchi, Balasubramanian Narasimhan, and Chiara Sabatti

Special thanks to: The National Science Foundation, Microsoft Cognitive Services

## Summary

Records of who owns, buys, and sells land in Kenya are sparse and, often, inaccessible. Journalists in Code for Africa’s network have named lack of transparency as a key barrier in investigating illegal and unjust land deals; and the World Resources Institute has identified lack of public land-ownership records as a barrier to influencing sustainable natural resource stewardship. 

This paper describes a process for extracting land records from a dataset of Kenyan government publications, the Kenya Gazettes. Our pipeline translates PDF data into raw text; extracts land-related notices; uses a Named Entity Recognition (NER) model to extract names of people, companies, addresses, and plots of land; and determines relationships between these entities, which we represent as a network. 

For segments published under the Land Registration Act, our NER model has strong performance (precision: 0.97 and recall: 0.91). Our network, constructed from 2,000+ gazettes published between 2000 and 2019, shows clear ownership records, many of which may have been previously unavailable. 

Our model demonstrates that the Kenya Gazettes are promising sources for structured and linked text data. Future work, led by Code for Africa (CfA), the World Resources Institute (WRI), and their partners, may apply the same pipeline to land notices that we were unable to process, announcements unrelated to land, or similarly-formulated publications from other countries. 


## Introduction

In 1996, billionaire Mike Maina, lawyer Chege Kirundi, and several Kenyan government officials allocated 45.5 acres of government trust land to a shell corporation associated with Kirundi and Maina. The company then sold that land back to the government and, shortly after, dissolved. In all, the deal lost Kenya’s National Social Security Fund, a core of the country’s social safety net, nearly Sh293 million.<sup>1</sup> Journalists associated with Code for Africa helped investigate and report this story in 2019, leading to a legal investigation and a return of this wealth, over 20 years later. 

This is not an isolated incident. Since colonization, elites, foreign corporations, and government officials have appropriated acres of land from public trust, private ownership, and expired leases. Today, Transparency International, an anti-corruption watchdog group, has ranked the lands ministry as “one of the most corrupt institutions in Kenya.<sup>2</sup> The 2004 Ndung’u Report, from the Commission of Inquiry into the Illegal/Irregular Allocation of Public land, called for mass repossession, investigation, and prosecution.<sup>3</sup>

Land corruption in Kenya is rooted in complex political and economic conditions, and access to more data alone will not alter these. That said, as the results of the Maina/Kirundi case suggests, legal pathways exist for returning land and wealth gained through unjust land deals; but building a successful case requires evidence. Journalists in Code for Africa’s network have named a lack of data about land -- and who buys, sells, owns, leases, and uses it -- as a barrier to uncovering illegal and unjust land deals. In particular, journalists have expressed a desire for **linked and aggregated data**, which, unlike keyword searches alone, can reveal patterns, connections, and outliers and act as a starting point for investigation.


### Our Contribution

Kenyan transparency laws require notices issued to the public in the weekly government Gazette, and these notices sometimes contain information about Kenya’s land, who owns or leases it, or how it is used.<sup>4</sup> These data are currently buried in thousands of pages of PDFs, searchable by keyword but not aggregated or linked. CfA and WRI believe that the Gazettes contain information that, if extracted and structured, can help to fill some of the gaps that currently exist. 

This paper describes our pipeline for building a dataset of land records from the Gazettes, focusing on Land Registration Act notices from 2012-2019. Beginning with PDFs, scanned and uploaded by CfA’s partners, we describe our process for developing text files, dividing these files into segments to isolate Land Registration Act announcements, extracting entities from these announcements (names, addresses, land plots, etc.), and mapping connections between these entities as a network. 

Our results demonstrate that the formulaic nature of Gazette announcements lends well to modern character recognition and Natural Language Processing (NLP) tools, and that the resulting dataset offers previously inaccessible insights. In particular, aggregated and networked information reveals patterns and outliers, which a keyword search cannot; and more specific and accurate entity extraction allows for faster connections. The Gazettes, and similar publications in other countries, remain a rich and largely-unlocked data source; and CfA engineers hope to apply a similar pipeline to other announcements. 

Our code is open source and available on a [Github repository](https://github.com/StanfordDataScience/dssg-cfa-public) for others to reuse and extend. Our dataset of PDF-to-text translations of the 2000-2019 Gazettes will be available through CfA’s databases, Connected Africa and Gazeti.Africa. 

## Methodology and Pipeline

Our project can be thought of as one continuous pipeline which starts with scans of the Kenya gazettes in PDF format available online and ends with a relational dataset connecting entities within these gazettes such as people, organizations, and plots of land. The pipeline can be split fairly neatly into four sections, as pictured above. A brief description of all four sections is written below:

* **Convert PDF to Raw Text:** Craft API calls to websites containing the Kenya Gazettes. Use Microsoft Cognitive Services' Read API on these PDFs. Finally, process json output of Read API into ordered text.

* **Divide Raw Text by Notice:** Extract separate segments from ordered text. Use regular expressions as a baseline for entity extraction.

* **Extract Entities From Notice** Build a spaCy model to extract entities more accurately. 

* **Build Relationships Between Entities** Using our spaCy model, extract entities for a number of gazettes. Combine these entities into objects, and these objects into a network. Visualize the network and perform exploratory data analysis.


Our outputs are: 
* A dataset of higher quality PDF-to-text translations of Kenya Gazettes
* A beginning dataset of entities extracted from 2000-2019 land records (names, addresses, land plots, etc.), stored in a data schema comparable to that already implemented within Aleph, CfA’s data journalism platform. 
* A network of these entities. 
* Rigorous documentation, with an aim of reproducibility 

![Project Pipeline Graphic](util/project_pipeline.png)

### Data Source

Our pipeline extracts text data from **Land Registration Act Notices issued in the Kenya Government Gazette**. We processed all publicly-available Gazettes published between 2000 and 2019; however, since we trained our entity recognition model on L.R.A. notices, nearly all of our data is from **after 2012**, the year the act was passed. We focused only on one segment due to time constraints, and we chose the L.R.A. because we had the largest number of sample notices for it and because each notice contains significant landownership information. 

The Kenya Gazette is a bulletin published by the Kenyan government. The weekly newspaper -- alongside occasional Special Issues -- contains notices to the public, mandated by transparency and governance laws. A sample table of contents shows announcements required by The State Corporations Act, The Mining Act, The Land Registration Act, and more. Though our prototype only works with notices related to the Land Registration Act, it is worth noting the extensive information that exists in this data source: 

![Gazette Table of Contents](util/table_of_contents.png)
*The Kenya gazettes contain a diverse set of information*

Each Gazette contains several *Notices*, generally mandated by a transparency law. Several of these relate to land, including The Land Registration Act, The Registered Land Act (in pre-2012 Gazettes), The Land Act, The National Land Commission, and the Environmental Management and Conservation Act.<sup>4</sup>. Sometimes, information about land ownership, transactions, and use exist within these Notices. 

![LRA Example](util/LRA_ex.png)
*A single Gazette Notice, as mandated by the Land Registration Act*

The Kenya Law Reporting Project has already scanned and uploaded publicly available Gazettes published between 1906 and 2019 to two online databases<sup>5, 6</sup>. We believe these databases, when combined, to be nearly comprehensive. 

Due to the scope of our project, we were only able to fully process notices mandated by the Land Registration Act and only in more recent Gazettes. We converted to text and tested our entity recognition model on all 2,261 Gazettes published between 2000 and 2019, in alignment with CfA journalists' perspectives that the most actionable data comes from the past two decades. However, since the Land Registration Act was passed in 2012, in practice, most of the data that makes it through our full pipeline is from after that year. 


![2000-2019 Gazettes](util/year_distr_2000_2019.png)
![LRA Distribution](util/year_distr_LRA.png)
![LRA Distribution Total](util/year_distr_LRA_notices.png)

## Pipeline

### Step 1: Converting PDF to Raw Text

#### 1a. Original text output: Tesseract

Our original dataset contained text files generated through an open-source PDF-to-text tool, Tesseract. Unfortunately, Tesseract’s character recognition did not allow for relation extraction. In particular, **Tesseract’s inability to separate columns of text resulted in a disjointed, irregularly-ordered output**. (In a random sampling from 30 gazettes, 56% of pages contained this error.) Without an automated way of isolating individual land announcements from each other, it was impossible to intuit which entities appeared in the same sections and, thus, the relationships between them -- the core of useful insights. 

A detailed report on our work with the Tesseract text can be found in the `util` folder of our [GitHub repo](https://github.com/StanfordDataScience/dssg-cfa-public). 

We believe that any relation extraction work will require ordered, higher-quality text. 

#### 1b. New text output: Microsoft Cognitive Services

Unable to meet project goals with Tesseract-generated text, we used Microsoft Cognitive Services’ Read API<sup>7</sup> to generate more reliable text files from PDFs stored on Connected Africa and Gazeti.Africa. More detail regarding accessing PDFs from the online databases and interacting with Microsoft’s API may be found on our GitHub. 

The Read API outputs a JSON data structure which contains, for each identified line, the text on that line, the coordinates of where that line is located on the page, and confidence ratings for each word in the line. Identified lines include reliable column separation. 

The primary limitation of the Read API itself is that it does not identify tables. Since the information relevant to our project is nearly all in paragraph form, we used the Read API because of its lower cost and equivalent performance on text. However, significant information exists in tables that may be valuable for future projects with different scopes.<sup>8</sup>


#### 1c. Read API JSON Output to Raw Text

We processed this Read API output, turning text and geometric information into ordered paragraphs using a k-means algorithm to identify columns and concatenating text. Our approach clusters the left side of the bounding boxes by x coordinate, increases the number of centroids to a relatively stable state, and uses the minimum number of centroids to represent the columns on the page. 

This approach reliably identifies two-column pages with paragraphs of text, which make up much of the Kenya gazettes. It occasionally fails on two-column pages with embedded tables and more often on tables with more than 4 columns.



<img src="util/pdf_scan.png" alt="PDF scan" width="200"/> 
<p style="text-align:center;">  PDF scan (full page) </p>
<img src="util/bounding_boxes.png" alt="Bounding Boxes" width="300"/>
<p style="text-align:center;">Bounding Boxes (full page)</p>
<img src="util/raw_text.png" alt="Raw Text" width="200"/> 
<p style="text-align:center;">Raw Text (portion of the page)</p>
    
  
### Step 2: Raw Text to Isolated Segments

The above page has eight segments related to land, each of which is a stand-alone and self-contained notice. Our later relation extraction relies on analyzing each of these 'segments,' as we will refer to them, in isolation. 

The highly structured nature of the Kenya Gazettes allowed us to apply regular expressions to determine the coordinates at which a segment begins and ends. 

The primary limitation is that we sometimes lose segments that span multiple pages, when a segment begins on a two-column page, like the one above, and ends on a page that includes tables or vice cersa. We err on the side of caution, and throw away segments which we suspect get corrupted by being read over multiple pages

<img src="util/table_and_text_pg.png" alt="Page with Table and Text" width="200"/> 

Even though this page contains neat paragraphs in the bottom half, it won’t get read because our scripts won’t be sure how many columns it has. It gets confused by the table on top mixed with paragraphs on the bottom.

### Step 3: Land Segments to Structured Data: Extracting Named Entities


### Step 4: Structured Data to Visualizing Connections: Building the Network


## Ethical Considerations

## Conclusion

## About

**Code for Africa (CfA)** is the largest civic technology, open data, and data journalism initiative on the African continent. 

The group is more a distributed network than a centralized organization. Core staff anchor a coalition of civic data and civic technology organizations, as well as an involved membership of individuals — “journalists, technologists, data scientists, and change makers”. 

Read more on [Medium](https://medium.com/code-for-africa), [GitHub](https://github.com/CodeForAfrica/), [Twitter](https://twitter.com/Code4Africa), and [Facebook](https://www.facebook.com/CodeForAfrica). 

**The World Resources Institute (WRI)** is a global research organization developing policy and practice for sustainable natural resource governance. Local satellites work with corporate and government leaders to reshape land, water, and climate management — with the mission of seeding an environment able to provide for the needs and aspirations of current and future generations. 

Read more on the [WRI website](https://www.wri.org), including [this page](https://www.wri.org/our-work/topics/africa) describing their Africa-based offices. 

**The Data Science + Social Good Program** at Stanford (DSSG) partners teams of undergraduate and graduate students with governments and non-profits to implement data science projects. The goals of the program are, first, to produce impactful data science projects; and, second, to train aspiring researchers, with the support of advanced graduate student and faculty mentors. 

DSSG runs for 8 weeks during the summer. The program is anchored at Stanford’s Data Science Institute, in network with similar programs at peer institutions, and has received generous support from the National Science Foundation. Read more on the DSSG [website](https://dssg.stanford.edu).

**Student fellows**

As of August 2020,

*Tsion (T) Tesfaye* is pursuing a master’s degree in statistics (data science track) at Stanford. She is particularly interested in the intersection of data science and design thinking to build innovative systems for purpose-driven companies. Her hobbies include reading autobiographies, exploring post-impressionist paintings and listening to country music.

*Thea Rossman* is an incoming master’s student in Computer Science at Stanford, where she just finished a B.A.S. in Mathematical & Computational Science and Ethnic Studies. She is interested how anti-oppression frameworks can influence public data systems. She loves speculative fiction and fantasy books, math education, and youth organizing.

*Robbie Thompson* is a junior at Stanford University studying Mathematical and Computational Science. He is particularly interested in quantitative methods in social science. Robbie’s hobbies include barefoot running and pickup basketball.

## References

[1]: Joseph Wangui, “NSSF loses Sh293m in shady land deal done 24 years ago,” *The Daily Nation*, January 4, 2020.

[2]: Katy Migiro, “Kenya to ‘purge’ land ministry of corrupt cartels: official,” *Reuters*, October 12, 2016; Republic of Kenya Ministry of Environment and Forestry, *Taskforce Report on Forest Resources Management and Logging Activities in Kenya*, 2018.

[3]: Ndungu Commission, *Report of the Commission of Inquiry into the Illegal/Irregular Allocation of Public Land*, Nairobi, Kenya, June 30, 2003. National Land Commission of the Republic of Kenya, “FAQ’s”, Accessed 8/20/2020. https://www.landcommission.go.ke/article/faqs

[4]: For a list of relevant land laws and the Gazette notices associated, see this document compiled by CfA: [“CfA, iLab, Stanford, Documentation, Land Legislation”](https://docs.google.com/document/d/1d-XGtYYbDdDFWph5ap_hMBsSKSfyCgRvuy5wcZvYihA/).]

[5]: National Council for Law Reporting (Kenya Law), *Connected Africa*. https://data.connectedafrica.net/

[6]: OpenGazettes Universe, *gazeti.Africa*. https://gazeti.africa

[7]: Documentation for the Read API may be found [here](https://docs.microsoft.com/en-us/azure/cognitive-services/computer-vision/concept-recognizing-text).

[8]: Though we did not implement it within our prototype, we have included instructions on our GitHub for using the Form Recognizer Layout API, also from Microsoft, for reliably extracting information from tables in the Gazettes. We believe that this will be necessary for reliably extracting information from tables, though there may be a strategy that we have not considered to create tables from the Read API output. Documentation for the Form Recognizer Layout API may be found [here](https://docs.microsoft.com/en-us/azure/cognitive-services/form-recognizer/overview).